# Concept induction
This notebook offers sample [EDSL](https://docs.expectedparrot.com) code for using language models to identify concepts in unstructured texts, then generate criteria for the concepts, and then apply the criteria to evaluate the texts.

This idea is inspired by the recent paper: [Concept Induction: Analyzing Unstructured Text with High-Level Concepts Using LLooM](https://hci.stanford.edu/publications/2024/Lam_LLooM_CHI24.pdf)

## Technical setup
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Identify concepts
We start by creating a general question prompting the respondent (a language model) to identify concepts in a given text.

EDSL comes with a variety of [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model. `QuestionList` may be appropriate where we want the response to be formatted as a list of strings:

In [1]:
from edsl import QuestionList

q_concepts = QuestionList(
    question_name="concepts",
    question_text="Identify the key concepts in the following text: {{ scenario.text }}",
    # max_list_items = # Optional
)

We might also want to ask some other questions about our data at the same time (a data labeling task). For example:

In [2]:
from edsl import QuestionMultipleChoice

q_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="Identify the sentiment of this text: {{ scenario.text }}",
    question_options=["Negative", "Neutral", "Positive"],
)

We parameterize the questions in order to run them for each of our texts. This is done with `Scenario` objects that we create for our data (here, some recent tweets by Pres. Biden):

In [3]:
# Replace with your data
texts = [  # POTUS recent tweets
    "Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",
    "We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",
    "Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.",
    "Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.",
    "This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'",
    "The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.",
    "Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day.",
    "Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.",
    "Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients.",
    "With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with.",
]
len(texts)

10

In [4]:
from edsl import ScenarioList

scenarios = ScenarioList.from_list("text", texts)
scenarios

,text
0,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C."
1,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom."
2,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.
3,"Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States."
4,"This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'"
5,The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.
6,"Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day."
7,Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.
8,"Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients."
9,"With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with."


Next we combine the questions into a survey in order to administer them together (asynchronously by default, or according to any skip/stop rules or other logic that we want to add--learn more about `Survey` methods in our [documentation](https://docs.expectedparrot.com/en/latest/surveys.html)):

In [5]:
from edsl import Survey

survey = Survey(questions=[q_concepts, q_sentiment])

We add the scenarios to the survey and then run it to generate a dataset of results:

In [6]:
results = survey.by(scenarios).run()

Job UUID,1b543322-4e03-40d6-9040-c97cbe54fb5b
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/1b543322-4e03-40d6-9040-c97cbe54fb5b
Exceptions Report URL,None
Results UUID,007a0f6d-5eb9-4bef-bedd-3cc74472d644
Results URL,https://www.expectedparrot.com/content/007a0f6d-5eb9-4bef-bedd-3cc74472d644


EDSL comes with [built-in methods](https://docs.expectedparrot.com/en/latest/results.html) for working with results</a> in a variety of forms (data tables, SQL queries, dataframes, JSON, CSV). We can call the `columns` method to see a list of all the components that we can analyze:

In [7]:
results.columns

,0
0,agent.agent_index
1,agent.agent_instruction
2,agent.agent_name
3,answer.concepts
4,answer.sentiment
5,cache_keys.concepts_cache_key
6,cache_keys.sentiment_cache_key
7,cache_used.concepts_cache_used
8,cache_used.sentiment_cache_used
9,comment.concepts_comment


We can select and print specific components to inspect in a table:

In [8]:
results.select("text", "concepts", "sentiment")

,scenario.text,answer.concepts,answer.sentiment
0,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.","['keynote address', 'U.S. Holocaust Memorial Museum', 'Annual Days of Remembrance', 'Washington, D.C.']",Neutral
1,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.","['nation of immigrants', 'nation of dreamers', 'Cinco de Mayo', 'freedom']",Positive
2,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,"['Medicare', 'Social Security', 'economic plan', 'Medicare solvency', 'Social Security solvency', 'fair share']",Positive
3,"Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.","['Army Black Knights', 'West Point', 'Commander-in-Chief Trophy', 'United States']",Positive
4,"This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'","['Holocaust Remembrance Day', 'six million Jews', 'Nazis', 'darkest chapters', 'lessons of the Shoah', 'responsibility', 'Never Again']",Neutral
5,The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.,"['Presidential Medal of Freedom', 'faith in freedom', ""America's faith"", 'better tomorrow']",Positive
6,"Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day.","['Teaching', 'Calling', 'Educator', 'White House', 'Teacher State Dinner']",Positive
7,Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.,"['Jill and I', 'warm wishes', 'Orthodox Christian communities', 'Easter', 'Lord bless and keep you', 'Easter Sunday', 'year ahead']",Positive
8,"Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients.","['Dreamers', 'health care', 'Affordable Care Act', 'DACA recipients', 'affordable health coverage', 'Administration']",Positive
9,"With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with.","['American comeback', '175,000 new jobs', 'Congressional Republicans', 'cut taxes', 'billionaires', 'special interests', 'job creation', 'economy', 'families']",Positive


If our concepts lists are too long, we can run another question prompting a model to condense it. We can specify the number of concepts that we want to get:

In [9]:
# Flattening our list of lists for all the texts to use in a follow-on question:
concepts_list = results.select("concepts").to_list(flatten=True)
concepts_list

['keynote address',
 'U.S. Holocaust Memorial Museum',
 'Annual Days of Remembrance',
 'Washington, D.C.',
 'nation of immigrants',
 'nation of dreamers',
 'Cinco de Mayo',
 'freedom',
 'Medicare',
 'Social Security',
 'economic plan',
 'Medicare solvency',
 'Social Security solvency',
 'fair share',
 'Army Black Knights',
 'West Point',
 'Commander-in-Chief Trophy',
 'United States',
 'Holocaust Remembrance Day',
 'six million Jews',
 'Nazis',
 'darkest chapters',
 'lessons of the Shoah',
 'responsibility',
 'Never Again',
 'Presidential Medal of Freedom',
 'faith in freedom',
 "America's faith",
 'better tomorrow',
 'Teaching',
 'Calling',
 'Educator',
 'White House',
 'Teacher State Dinner',
 'Jill and I',
 'warm wishes',
 'Orthodox Christian communities',
 'Easter',
 'Lord bless and keep you',
 'Easter Sunday',
 'year ahead',
 'Dreamers',
 'health care',
 'Affordable Care Act',
 'DACA recipients',
 'affordable health coverage',
 'Administration',
 'American comeback',
 '175,000 new

In [10]:
q_condense = QuestionList(
    question_name="condense",
    question_text="Return a condensed list of the following list of concepts: "
    + ", ".join(concepts_list),
    max_list_items=10,
)

Note that we can call the `run()` method on either a survey of questions or an individual question:

In [11]:
results = q_condense.run()

Job UUID,db025082-304f-4ecc-b200-f242dc4ef866
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/db025082-304f-4ecc-b200-f242dc4ef866
Exceptions Report URL,None
Results UUID,904f5917-9159-4ce1-bf17-65a5b904080c
Results URL,https://www.expectedparrot.com/content/904f5917-9159-4ce1-bf17-65a5b904080c


In [12]:
results.select("condense")

,answer.condense
0,"['Holocaust Remembrance', 'Nation of Immigrants', 'Freedom and Social Programs', 'Economic Plan and Job Creation', 'U.S. Military and Honors', 'Education and Teaching', 'Faith and Easter', 'American Comeback', 'Health Care and DACA', 'Taxation and Special Interests']"


## Identify criteria for each concept
Similar to our first step, next we can run a question prompting the model to generate criteria for each concept. We could use `QuestionFreeText` to generate criteria in an unstructured narrative:

In [13]:
from edsl import QuestionFreeText

q_criteria = QuestionFreeText(
    question_name="criteria",
    question_text="""Describe key criteria for determining whether a text is primarily about the 
    following concept: {{ scenario.concept }}""",
)

For this question, the scenarios are the concepts that we generated:

In [14]:
condensed_concepts_list = results.select("condense").to_list(flatten=True)

scenarios = ScenarioList.from_list("concept", condensed_concepts_list)
scenarios

,concept
0,Holocaust Remembrance
1,Nation of Immigrants
2,Freedom and Social Programs
3,Economic Plan and Job Creation
4,U.S. Military and Honors
5,Education and Teaching
6,Faith and Easter
7,American Comeback
8,Health Care and DACA
9,Taxation and Special Interests


In [15]:
results = q_criteria.by(scenarios).run()

Job UUID,2eb2d6aa-4741-47bc-a26f-b4d74822d3bb
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/2eb2d6aa-4741-47bc-a26f-b4d74822d3bb
Exceptions Report URL,None
Results UUID,2d5e7451-7448-4b5b-bcc0-32a1453224fe
Results URL,https://www.expectedparrot.com/content/2d5e7451-7448-4b5b-bcc0-32a1453224fe


In [16]:
results.select("concept", "criteria")

,scenario.concept,answer.criteria
0,Holocaust Remembrance,"Determining whether a text is primarily about Holocaust Remembrance involves evaluating several key criteria: 1. **Subject Focus**: The text should center on the Holocaust, specifically emphasizing the importance of remembering the events, victims, and survivors. It should discuss commemorative practices, memorials, or anniversaries related to the Holocaust. 2. **Purpose and Intent**: The text's purpose should be to educate, memorialize, or promote awareness of the Holocaust. It may aim to prevent future atrocities through remembrance and reflection on past events. 3. **Language and Tone**: The language used should convey a sense of solemnity, respect, and reflection. The tone might be educational, commemorative, or advocacy-oriented, urging readers to remember and learn from the past. 4. **Content and Themes**: Key themes should include memory, history, education, and the moral and ethical lessons of the Holocaust. The text might discuss survivor testimonies, historical accounts, or the impact of Holocaust education on contemporary society. 5. **Contextual References**: The text may reference specific Holocaust Remembrance events, such as Yom HaShoah (Holocaust Remembrance Day), International Holocaust Remembrance Day, or other national and international commemorations. 6. **Audience Engagement**: The text might encourage reader participation in remembrance activities, such as attending memorial services, visiting Holocaust museums, or engaging in educational programs. 7. **Historical Accuracy**: The text should be grounded in historical facts about the Holocaust, providing accurate information about the events, figures, and outcomes associated with it. 8. **Cultural and Social Impact**: The text should reflect on the broader cultural and social significance of remembering the Holocaust, including its impact on Jewish communities and global human rights discourse. By considering these criteria, one can assess whether a text is primarily focused on Holocaust Remembrance and its associated themes."
1,Nation of Immigrants,"Determining whether a text is primarily about the concept of a ""Nation of Immigrants"" involves evaluating several key criteria: 1. **Historical Context**: The text discusses the history of immigration in the country, emphasizing waves of immigrants from various regions and how they have shaped the nation’s development. 2. **Cultural Diversity**: It highlights the cultural contributions of different immigrant groups, showcasing the multicultural makeup of the nation and how these diverse cultures coexist and interact. 3. **Immigration Policy**: The text examines the laws, policies, and governmental attitudes towards immigration, including changes over time and their impact on the nation’s demographic and social landscape. 4. **Economic Impact**: Discussion of the economic roles that immigrants have played, such as filling labor shortages, starting businesses, and contributing to innovation and economic growth. 5. **Social Integration and Challenges**: The text addresses how immigrants have integrated into society, including challenges they face such as discrimination, assimilation, and identity issues. 6. **National Identity**: It explores the idea of national identity in the context of being a ""Nation of Immigrants,"" including debates about what it means to belong to the nation and how immigration shapes national values and identity. 7. **Personal Narratives and Case Studies**: The inclusion of personal stories or case studies of immigrants, illustrating their journeys, struggles, and successes, and how these individual experiences reflect broader trends. 8. **Symbolic References**: Use of symbols or references commonly associated with the concept, such as the Statue of Liberty in the United States, which is often linked to welcoming immigrants. 9. **Comparative Analysis**: The text may compare the nation in question with other countries regarding 

## Identify the concepts in each text and evaluate based on the criteria
Finally, we can use the concepts and the criteria to run another question where we prompt the model to evaulate each text. Question types `QuestionLinearScale`, `QuestionRank` or `QuestionNumerical` may be appropriate where we want to return a score:

In [17]:
from edsl import QuestionLinearScale

q_score = QuestionLinearScale(
    question_name="score",
    question_text="""Consider the following concept and criteria for determining whether 
    a given text addresses this concept. Then score how well the following text satisfies
    the criteria for the concept.
    Concept: {{ scenario.concept }}
    Criteria: {{ scenario.criteria }}
    Text: {{ scenario.text }}""",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    option_labels={0: "Not at all", 10: "Very well"},  # Optional
)

Here we want to use both the texts and the concepts and corresponding criteria together as scenarios of the question:

In [18]:
concepts_criteria = [
    list(pair)
    for pair in zip(
        results.select("concept").to_list(), results.select("criteria").to_list()
    )
]
len(concepts_criteria)

10

In [19]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList(
    Scenario({"text": text, "concept": concept, "criteria": criteria})
    for text in texts
    for [concept, criteria] in concepts_criteria
)

In [20]:
results = q_score.by(scenarios).run()

Job UUID,8efa3e22-3212-4ce2-9074-c87f9df846d2
Progress Bar URL,https://www.expectedparrot.com/home/remote-job-progress/8efa3e22-3212-4ce2-9074-c87f9df846d2
Exceptions Report URL,None
Results UUID,1c730d8e-b4d4-4ab0-b19f-5ff084b37a99
Results URL,https://www.expectedparrot.com/content/1c730d8e-b4d4-4ab0-b19f-5ff084b37a99


We can filter the results based on the responses--e.g., here we just show the non-zero scores:

In [21]:
(
    results.filter("score > 0")
    .select("text", "concept", "score")
)

,scenario.text,scenario.concept,answer.score
0,"Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",Holocaust Remembrance,5
1,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",Nation of Immigrants,1
2,"We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",Freedom and Social Programs,1
3,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,Freedom and Social Programs,3
4,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,Economic Plan and Job Creation,1
5,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,American Comeback,3
6,Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.,Taxation and Special Interests,3
7,"Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.",U.S. Military and Honors,3
8,"This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'",Holocaust Remembrance,9
9,The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.,Freedom and Social Programs,1


## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the scenarios, survey and results from above, and this notebook:

In [22]:
scenarios.push(description = "Example scenarios", alias = "example-scenarios", visibility = "public")

{'description': 'Example scenarios',
 'object_type': 'scenario_list',
 'url': 'https://www.expectedparrot.com/content/42b0d3de-e1f8-41e3-afe9-e8609745412c',
 'uuid': '42b0d3de-e1f8-41e3-afe9-e8609745412c',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

In [23]:
survey.push(description = "Example survey", alias = "example-survey", visibility = "public")

{'description': 'Example survey',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/82339abb-7763-425c-bf80-338a71cf1dcb',
 'uuid': '82339abb-7763-425c-bf80-338a71cf1dcb',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

In [24]:
results.push(description = "Example results", alias = "example-results", visibility = "public")

{'description': 'Example results',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/41bf351c-4900-449a-9c18-5840d04ec060',
 'uuid': '41bf351c-4900-449a-9c18-5840d04ec060',
 'version': '0.1.47.dev1',
 'visibility': 'public'}

We can also post this notebook:

In [25]:
from edsl import Notebook

In [28]:
nb = Notebook(path = "concept_induction.ipynb")

if refresh := False:
    nb.push(
        description = "Example code for concept induction", 
        alias = "concept-induction-notebook",
        visibility = "public"
    )
else:
    nb.patch('42642a6a-4ae0-4f37-960a-b8798987e923', value = nb)